In [ ]:
# THIS CODE IS FOR SINGLE z-SLICE LEVEL IBM PREDICTION

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset
from skimage import io
import os
import pandas as pd
from load_image_data import embhealth
import numpy as np
import sys

In [ ]:
# SETUP THE DEVICE

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(torch.cuda.current_device()))
CUDA_LAUNCH_BLOCKING=1
print(torch.__version__)

In [ ]:
# DEFINE PARAMETERS AND FILEPATHS

dropout_rate=0.3
base_folder="your system path to extracted embryo_ls_glim-main folder"
model_name="IBM"
f_num="healthy"
folder=os.path.join(base_folder,model_name) # path to pretrained IBM folder
folderw=f"{base_folder}/example_data/{f_num}" # path to write folder where IBM writes the result csv
if os.path.isdir(folderw):
    print("Directory already exists\n")
else:
    os.mkdir(folderw)
    print("Folder created\n")


In [ ]:
# DEFINE DATASET

test_dataset=embhealth(
    csv_file=f"{folderw}/overlapped/unknown.csv", # per slice label-file
    root_dir=f"{folderw}/overlapped",# image directory
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize([633,630]), # resize to match model input size  
    ])
 )

In [ ]:
# LOAD STANDARD PRETRAINED MODEL

model=torchvision.models.efficientnet_b7(pretrained=True)
#print(model)
#print(model, file=open(f"{folder}/saved_images/base_model.txt",'a'))

In [ ]:
# CHANGE MODEL ARCHITECTURE TO MATCH THE TRAINED MODEL

for param in model.parameters():
    param.requires_grad=False

model.classifier[0]=nn.Dropout(p=dropout_rate, inplace=True)
model.classifier[1]=nn.Sequential(
    nn.Linear(in_features=2560,out_features=500,bias=True),
    nn.ReLU(),
    nn.Linear(in_features=500,out_features=200,bias=True),
    nn.ReLU(),
    nn.Linear(in_features=200,out_features=2,bias=True),
    
)
model=model.to(device)
#print(model)
#print(model, file=open(f"{folder}/saved_images/modified_model.txt",'a'))

In [ ]:
# MAKE A REPLICA OF THE MODIFIED MODEL AND LOAD TRAINED MODEL PARAMETERS

model2=model
model2.load_state_dict(torch.load(f"{folder}/modelp99.pth"))
#print(model2)

In [ ]:
# MAKE PREDICTIONS ON THE TEST DATASET

model=model2 
classes = ["Healthy","Sick"]

model.eval()

selected_dataset=test_dataset

predictions=[]
probabilities=[]
prob_1=[]
true_label=[]
name=[]
print(f"Using {device} device")
with torch.no_grad():
    for i,j in enumerate(range(len(selected_dataset))):
        x1,y1=selected_dataset[j][0],selected_dataset[j][1]
        name.append(selected_dataset[j][2])
        x1= torch.unsqueeze(x1, 0)
        x1,y1 =x1.to(device), y1.to(device)
        pred=model(x1)
        predicted,actual=classes[pred[0].argmax(0)],classes[y1]
        m = torch.nn.Softmax(dim=0)
        prob=m(pred[0])
        probabilities.append(prob[prob.argmax(0)].cpu().numpy())
        prob_1.append(prob[1].cpu().numpy())
        true_label.append(y1.cpu().numpy())
        predictions.append(pred[0].argmax(0).cpu().numpy())

# write results to csv         
df = pd.DataFrame({'name':name,
                   'predicted': predictions,

                   'actual': true_label,
                   
                   'probability':probabilities,
                   'prob_1':prob_1
                  
                  }
                 )
df.to_csv(f"{folderw}/overlapped/check.csv")
ones=np.sum(predictions)
total=len(predictions)
sick=ones/total
healthy=(total-ones)/total

print(f'IBM predicts: Healthy:{healthy*100}%, sick {sick*100}%')